## 빅데이터 실습

### 다나와 무선청소기 상품 분석

#### 데이터 수집

##### 셀레니움 사용
- 무선청소기 검색에 사용할 URL
    'https://search.danawa.com/dsearch.php?query=무선청소기&originalQuery=무선청소기&previousKeyword=무선청소기&checkedInfo=N&volumeType=allvs&page=1&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'


In [25]:
# 필요 라이브러리 등록
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from urllib import parse  # url encode
from tqdm import tqdm

import time

In [26]:
## 웹드라이버로 크롬 오픈

url = r'https://search.danawa.com/dsearch.php?query=무선청소기&page=1&limit=120&sort=saveDESC'

driver = webdriver.Chrome()
driver.get(url)

time.sleep(2.0)

In [27]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [28]:
prodItems = soup.select('ul.product_list > li.prod_item')

In [29]:
# id = "productItem19965230" 식으로 전개되므로 productItem이 아니면 패스
# 상품명 가져오기, 만약 애드스마트 영역(HTML)이면 예외발생
prodItems[0].select('p.prod_name > a')[0].text

'LG전자 오브제컬렉션 코드제로 A9S AX9884'

In [30]:
# 상품 스펙 목록 가져오기
prodItems[0].select('div.spec_list')[0].text.strip().replace('\t', '').replace(' / ', ' ,')

'핸디스틱청소기 ,무선 ,흡입+물걸레(동시) ,흡입력: 250W ,소비전력: 620W ,2023년형 ,[구성] 거치대: 올인원타워 ,먼지비움 ,충전 ,UVC LED ,액서서리수납 ,스탠드거치 ,먼지비움시간: 30초 ,브러쉬: 와이드바닥 ,물걸레: 스팀, 고온, 일반 ,솔형 ,틈새 ,먼지봉투: 2.5L ,[배터리] 사용시간: 30분(최대) ,충전시간: 4시간 ,분리형(2개) ,리튬이온 ,25.2V ,[청소] 싸이클론흡입 ,LED라이트 ,자동물공급 ,스마트인버터모터 ,[부가] 스마트폰제어 ,대화형알림창 ,워셔블헤파필터 ,5단계여과 ,색상: 카밍베이지 ,무게: 2.47kg ,크기(가로x세로x깊이): 300x1120x245mm'

In [31]:
# 상품 가격 가져오기
# input 중 두번째 값이 최소값을 담고 있는 것을 확인
prodItems[0].select('input')[1].get('value')

'873560'

In [32]:
# 상품 인트로 가져오기
prodItems[0].select('p.intro_text')[0].text

'가장 프리미엄 라인업(AX)으로 스팀물걸레 포함 구성, AX9988보다 흡입력만 낮음'

#### 1번 페이지에서 상품 정보 추출

In [33]:
prodData = []
loop = 0
for prodItem in prodItems:
    try:
        prodName = prodItem.select('p.prod_name > a')[0].text.strip()
        specList = prodItem.select('div.spec_list')[0].text.strip().replace('\t', '')
        price = prodItem.select('input')[1].get('value')
        loop += 1
        prodData.append([prodName, specList, price])
    except:
        pass

In [34]:
# 결과 확인
len(prodData)

120

##### 여러페이지 크롤링 하는 방법

In [35]:
# 검색어, 페이지를 변경하면서 URL 생성함수
def getSearchPageUrl(keyword, page):
    encodekeyword = parse.quote(keyword)
    url = f'https://search.danawa.com/dsearch.php?query={encodekeyword}&originalQuery={encodekeyword}&previousKeyword={encodekeyword}&checkedInfo=N&volumeType=allvs&page={page}&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'
    return url

In [36]:
# 상품정보 추출하는 함수
def getProdItems(prodItems):
    prodData = []
    loop = 0
    for prodItem in prodItems:
        try:
            prodName = prodItem.select('p.prod_name > a')[0].text.strip()
            specList = prodItem.select('div.spec_list')[0].text.strip().replace('\t', '')
            price = prodItem.select('input')[1].get('value')
            loop += 1
            prodData.append([prodName, specList, price])
        except:
            pass
    
    return prodData

In [37]:
# 검색어에 ' '(공백)이 있으면 URL에서 링크 클릭이 안되는 문제(VSCode 기준) -> urlEncode : parse.quoto(keyword)
getSearchPageUrl('마이크로 sd카드', 5)

'https://search.danawa.com/dsearch.php?query=%EB%A7%88%EC%9D%B4%ED%81%AC%EB%A1%9C%20sd%EC%B9%B4%EB%93%9C&originalQuery=%EB%A7%88%EC%9D%B4%ED%81%AC%EB%A1%9C%20sd%EC%B9%B4%EB%93%9C&previousKeyword=%EB%A7%88%EC%9D%B4%ED%81%AC%EB%A1%9C%20sd%EC%B9%B4%EB%93%9C&checkedInfo=N&volumeType=allvs&page=5&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'

In [38]:
# 여러페이지 검색 후 크롤링하는 작업
driver = webdriver.Chrome()
# 암묵적으로 웹 자원 로드를 위해 3초정도 대기
driver.implicitly_wait(3.0)

keyword = '무선청소기'
totalPage = 10
prodDataTotal = []

for page in tqdm(range(1, totalPage+1)):
    url = getSearchPageUrl(keyword, page)
    driver.get(url)
    time.sleep(5)
    
    # 현재 페이지 HTML 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 상품정보 추출
    prodItems = soup.select('ul.product_list > li.prod_item')
    prodItemList = getProdItems(prodItems)
    
    # 추출된 정보를 prodDataTotal 추가
    prodDataTotal += prodItemList

100%|██████████| 10/10 [01:37<00:00,  9.74s/it]


In [39]:
dfProdData = pd.DataFrame(prodDataTotal)

In [40]:
dfProdData.columns = [['상품명', '스펙정보', '가격']]

In [41]:
dfProdData

,상품명,스펙정보,가격
0,LG전자 오브제컬렉션 코드제로 A9S AX9884,핸디스틱청소기 / 무선 / 흡입+물걸레(동시) / 흡입력: 250W / 소비전력: ...,873560
1,삼성전자 비스포크 제트 VS20B956AX,핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 220W / 2022년형 / [구성]...,446320
2,샤오미 미홀 M22,핸디스틱청소기 / 무선 / 흡입형 / 소비전력: 320W / 2023년형 / [구성...,151050
3,LG전자 오브제컬렉션 코드제로 A9S AX9604,핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 250W / 소비전력: 620W / ...,715570
4,LG전자 오브제컬렉션 코드제로 A9S AX9988,핸디스틱청소기 / 무선 / 흡입+물걸레(동시) / 흡입력: 280W / 소비전력: ...,1229240
...,...,...,...
1195,포엔코 PC-OP01,욕실청소기 / 무선 / 전동회전 / [배터리] 충전시간: 1시간 / 리튬이온 / 1...,43530
1196,앱코 오엘라 네오스핀 OBC-AW09 + OST-AWUVCGY,욕실청소기 / 무선 / 전동회전 / 회전속도: 220RPM / 소비전력: 8W / ...,84900
1197,삼성전자 제트 VS20T9257SECS,핸디스틱청소기 / 무선 / 흡입+물걸레(교체) / 흡입력: 200W / 소비전력: ...,1384300
1198,삼성전자 제트 VS20T9257SDCS,핸디스틱청소기 / 무선 / 흡입+물걸레(교체) / 흡입력: 200W / 소비전력: ...,1168880


In [42]:
dfProdData.tail()

,상품명,스펙정보,가격
1195,포엔코 PC-OP01,욕실청소기 / 무선 / 전동회전 / [배터리] 충전시간: 1시간 / 리튬이온 / 1...,43530
1196,앱코 오엘라 네오스핀 OBC-AW09 + OST-AWUVCGY,욕실청소기 / 무선 / 전동회전 / 회전속도: 220RPM / 소비전력: 8W / ...,84900
1197,삼성전자 제트 VS20T9257SECS,핸디스틱청소기 / 무선 / 흡입+물걸레(교체) / 흡입력: 200W / 소비전력: ...,1384300
1198,삼성전자 제트 VS20T9257SDCS,핸디스틱청소기 / 무선 / 흡입+물걸레(교체) / 흡입력: 200W / 소비전력: ...,1168880
1199,삼성전자 제트 VS20T9259SECS,핸디스틱청소기 / 무선 / 흡입+물걸레(교체) / 흡입력: 200W / 소비전력: ...,1879530


In [43]:
dfProdData.to_excel('./data/무선청소기_인기상품_분석결과.xlsx', index=False)